In [1]:
import psycopg2
import pandas as pd
from prophet import Prophet
import datetime
from sklearn.preprocessing import StandardScaler,OneHotEncoder

In [2]:
connection = psycopg2.connect(
    host = 'diasterdb2.cbyvxmyzmzyh.us-east-2.rds.amazonaws.com',
    port = 5432,
    user = 'postgres',
    password = 'group7_db',
    database='diasterdb'
    )
cursor=connection.cursor()

In [3]:
sql = """
SELECT "table_name","column_name", "data_type", "table_schema"
FROM INFORMATION_SCHEMA.COLUMNS
WHERE "table_schema" = 'public'
ORDER BY table_name  
"""
pd.read_sql(sql, con=connection)

,table_name,column_name,data_type,table_schema
0,FPIS,NAME,character varying,public
1,FPIS,FIPS_ID,integer,public
2,TEMPERATURE,DATE,date,public
3,TEMPERATURE,Country,character varying,public
4,TEMPERATURE,AverageTemperatureUncertaintyF,numeric,public
5,TEMPERATURE,STATE,character varying,public
6,TEMPERATURE,TEMPERATURE_ID,integer,public
7,TEMPERATURE,Fahrenheit,numeric,public
8,TEMPERATURE,AverageTemperatureF,numeric,public
9,USDisaster,pa_program_declared,integer,public


In [4]:
sql_USDisaster = """ SELECT * FROM public."USDisaster" ORDER BY declaration_date DESC """
disaster_df = pd.read_sql(sql_USDisaster, con=connection)
disaster_df

,fema_declaration,disaster_number,state,declaration_type,declaration_date,fy_declared,incident_type,declaration_title,ih_program_declared,ia_program_declared,pa_program_declared,hm_program_declared,incident_begin_date,fips,place_code,designated_area,declaration_request_number,hash,last_refresh,id
0,FM-5416-CA,5416,CA,FM,2021-08-25,2021,Fire,French Fire,0,0,1,1,2021-08-24,6029,99029,Kern (County),21088,b2120fdada11e2ceefe3714927afe2443c719812,2021-08-25,61266e5e434e34d5eee6f25e
1,DR-4610-CA,4610,CA,DR,2021-08-24,2021,Fire,Wildfires,1,0,1,1,2021-07-14,6063,99063,Plumas (County),21087,d615785ddbaca2e0dbb66805d84246680c86f560,2021-08-25,6125a937434e34d5ee5cd8af
2,DR-4610-CA,4610,CA,DR,2021-08-24,2021,Fire,Wildfires,1,0,1,1,2021-07-14,6057,99057,Nevada (County),21087,dde89f65556f220034ac16cfa66abf7d135962da,2021-08-25,6125a937434e34d5ee5cd8c1
3,DR-4610-CA,4610,CA,DR,2021-08-24,2021,Fire,Wildfires,1,0,1,1,2021-07-14,6035,99035,Lassen (County),21087,74f77c93f158659bf1dd9f871227a9993244478d,2021-08-25,6125a937434e34d5ee5cd8a3
4,DR-4610-CA,4610,CA,DR,2021-08-24,2021,Fire,Wildfires,1,0,1,1,2021-07-14,6061,99061,Placer (County),21087,6e1ab1c91a9838148684d35413b598c5ea66df71,2021-08-25,6125a937434e34d5ee5cd8b6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61801,DR-5-MT,5,MT,DR,1953-06-06,1953,Flood,Floods,0,1,1,1,1953-06-06,30000,0,Statewide,53006,954449c15634fb45c8bea3ac975782793ccde050,2021-07-13,60c3b7a9a0ee349d71025774
61802,DR-4-MI,4,MI,DR,1953-06-02,1953,Tornado,Tornado,0,1,1,1,1953-06-02,26000,0,Statewide,53004,87a0c1dd5da249767f545e0c0a43f917e4e9ca83,2021-07-13,60c3b7a9a0ee349d7102577a
61803,DR-3-LA,3,LA,DR,1953-05-29,1953,Flood,Flood,0,1,1,1,1953-05-29,22000,0,Statewide,53005,4fb19699fdbba1387ffa2263fcc4a4e37a1de6d6,2021-07-13,60c3b7a9a0ee349d71025777
61804,DR-2-TX,2,TX,DR,1953-05-15,1953,Tornado,Tornado & Heavy Rainfall,0,1,1,1,1953-05-15,48000,0,Statewide,53003,c879557e78d059e6847e7688388c339d10f51979,2021-07-13,60c3b7a9a0ee349d71025783


In [5]:
sql_temp = """ SELECT * FROM public."TEMPERATURE" ORDER BY "TEMPERATURE_ID" ASC """
temp_df = pd.read_sql(sql_temp, con=connection)

# Prophet Disaster Prediction

In [6]:
disaster_df.dtypes

fema_declaration              object
disaster_number                int64
state                         object
declaration_type              object
declaration_date              object
fy_declared                    int64
incident_type                 object
declaration_title             object
ih_program_declared            int64
ia_program_declared            int64
pa_program_declared            int64
hm_program_declared            int64
incident_begin_date           object
fips                           int64
place_code                    object
designated_area               object
declaration_request_number     int64
hash                          object
last_refresh                  object
id                            object
dtype: object

In [7]:
# change incident begin date to datetime format
disaster_df["incident_begin_date"]= pd.to_datetime(disaster_df["incident_begin_date"])
disaster_df.dtypes

fema_declaration                      object
disaster_number                        int64
state                                 object
declaration_type                      object
declaration_date                      object
fy_declared                            int64
incident_type                         object
declaration_title                     object
ih_program_declared                    int64
ia_program_declared                    int64
pa_program_declared                    int64
hm_program_declared                    int64
incident_begin_date           datetime64[ns]
fips                                   int64
place_code                            object
designated_area                       object
declaration_request_number             int64
hash                                  object
last_refresh                          object
id                                    object
dtype: object

In [8]:
# filter dataframe to only diasters in TX
df_filtered = disaster_df[disaster_df["state"] == "TX"]
df_filtered.head()

,fema_declaration,disaster_number,state,declaration_type,declaration_date,fy_declared,incident_type,declaration_title,ih_program_declared,ia_program_declared,pa_program_declared,hm_program_declared,incident_begin_date,fips,place_code,designated_area,declaration_request_number,hash,last_refresh,id
604,DR-4586-TX,4586,TX,DR,2021-02-19,2021,Severe Ice Storm,Severe Winter Storms,1,0,1,1,2021-02-11,48477,99477,Washington (County),21013,9c8705c29e0c7daf5af7a42fca3beae6313afbd6,2021-07-13,60c3b874a0ee349d710be84c
605,DR-4586-TX,4586,TX,DR,2021-02-19,2021,Severe Ice Storm,Severe Winter Storms,1,0,1,1,2021-02-11,48499,99499,Wood (County),21013,e9d17ea39bc60d70fde74bde5ebf2a9ba8f5519b,2021-07-13,60c3b874a0ee349d710be888
606,DR-4586-TX,4586,TX,DR,2021-02-19,2021,Severe Ice Storm,Severe Winter Storms,1,0,1,1,2021-02-11,48497,99497,Wise (County),21013,61441a1eea4446ff22b348d0ed6c34d81932e570,2021-07-13,60c3b874a0ee349d710be883
607,DR-4586-TX,4586,TX,DR,2021-02-19,2021,Severe Ice Storm,Severe Winter Storms,1,0,1,1,2021-02-11,48471,99471,Walker (County),21013,182253f33c1ef75b0be48effc7b987c1a9d1c05b,2021-07-13,60c3b874a0ee349d710be877
608,DR-4586-TX,4586,TX,DR,2021-02-19,2021,Severe Ice Storm,Severe Winter Storms,1,0,1,1,2021-02-11,48493,99493,Wilson (County),21013,458456f7492cc9613a9a537388c23dc427676aa7,2021-07-13,60c3b874a0ee349d710be876


In [9]:
# filter so that we remove any biological disaster
df_filtered2 = df_filtered[df_filtered["incident_type"] != "Biological"]
len(df_filtered2)

4593

In [10]:
# group dataframe by incident begin date
df_groupby = df_filtered2.groupby("incident_begin_date").count()
df_groupby.head()

,fema_declaration,disaster_number,state,declaration_type,declaration_date,fy_declared,incident_type,declaration_title,ih_program_declared,ia_program_declared,pa_program_declared,hm_program_declared,fips,place_code,designated_area,declaration_request_number,hash,last_refresh,id
incident_begin_date,,,,,,,,,,,,,,,,,,,
1953-05-15,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1953-06-19,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1954-07-01,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1957-04-29,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1958-06-06,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [11]:
# check values in incident type
df_groupby["incident_type"]

incident_begin_date
1953-05-15      1
1953-06-19      1
1954-07-01      1
1957-04-29      1
1958-06-06      1
             ... 
2019-08-16      1
2019-09-17      7
2020-07-25     32
2020-08-23     67
2021-02-11    508
Name: incident_type, Length: 273, dtype: int64

In [12]:
# reset index
df_groupby = df_groupby.reset_index()

In [13]:
# set values for data and headers for prophet df
data = [df_groupby["incident_begin_date"], df_groupby["incident_type"]]
headers=["ds", "y"]

In [14]:
# create prophet DF
df_prophet = pd.concat(data, axis=1, keys=headers)
df_prophet.head()

,ds,y
0,1953-05-15,1
1,1953-06-19,1
2,1954-07-01,1
3,1957-04-29,1
4,1958-06-06,1


In [15]:
# check dtypes
df_prophet.dtypes

ds    datetime64[ns]
y              int64
dtype: object

In [16]:
# fit model
m = Prophet()
m.fit(df_prophet)

INFO:numexpr.utils:NumExpr defaulting to 4 threads.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [17]:
# generate future periods
future = m.make_future_dataframe(periods=365)
future.tail()

,ds
633,2022-02-07
634,2022-02-08
635,2022-02-09
636,2022-02-10
637,2022-02-11


In [18]:
# predict values for future periods
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]


,ds,yhat,yhat_lower,yhat_upper
0,1953-05-15,-16.123552,-76.908789,44.828634
1,1953-06-19,-17.945645,-82.539927,38.735934
2,1954-07-01,-13.510487,-77.756658,52.204475
3,1957-04-29,-5.545605,-67.723798,55.699047
4,1958-06-06,-11.558757,-73.750606,51.961436
...,...,...,...,...
633,2022-02-07,97.145580,35.074975,162.828537
634,2022-02-08,92.784941,30.488995,152.168313
635,2022-02-09,89.796345,27.053690,149.278317
636,2022-02-10,98.665272,33.319276,161.838673


In [28]:
# plot interactive version of graph
from prophet.plot import plot_plotly, plot_components_plotly

fig = plot_plotly(m, forecast)

In [32]:
import plotly.express as px

fig.write_html("disaster.html")
fig

# Prophet Temperature Prediction

In [20]:
# change date to datetime
temp_df["DATE"]= pd.to_datetime(temp_df["DATE"])
temp_df.dtypes

TEMPERATURE_ID                             int64
DATE                              datetime64[ns]
STATE                                     object
Country                                   object
Fahrenheit                               float64
AverageTemperatureF                      float64
AverageTemperatureUncertaintyF           float64
dtype: object

In [21]:
# filter data frame to NY
temp_filtered = temp_df[temp_df["STATE"] == "New York"]


In [22]:
# sort by date
temp_sorted = temp_filtered.sort_values("DATE")
temp_sorted.head()

,TEMPERATURE_ID,DATE,STATE,Country,Fahrenheit,AverageTemperatureF,AverageTemperatureUncertaintyF
23326,23327,1953-01-01,New York,United States,10.571,27.7142,32.3546
23327,23328,1953-02-01,New York,United States,12.524,28.1048,32.2610
23328,23329,1953-03-01,New York,United States,47.228,35.0456,32.1494
23329,23330,1953-04-01,New York,United States,90.365,43.6730,32.3996
23330,23331,1953-05-01,New York,United States,158.153,57.2306,32.5544


In [23]:
# assign values to prohet df 
data2 = [temp_sorted["DATE"], temp_sorted["AverageTemperatureF"]]
headers2=["ds", "y"]

In [24]:
# create prophete DF
df_prophet2 = pd.concat(data2, axis=1, keys=headers2)
df_prophet2.head()

,ds,y
23326,1953-01-01,27.7142
23327,1953-02-01,28.1048
23328,1953-03-01,35.0456
23329,1953-04-01,43.6730
23330,1953-05-01,57.2306


In [25]:
# fit model and apply seasonality of monthly occurances
m2 = Prophet(seasonality_mode='multiplicative').fit(df_prophet2)
future2 = m2.make_future_dataframe(periods=120, freq='MS')
forecast2 = m.predict(future2)
forecast2[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,ds,yhat,yhat_lower,yhat_upper
0,1953-01-01,-23.742800,-87.170803,35.393951
1,1953-02-01,46.959410,-10.744473,108.619864
2,1953-03-01,-18.460167,-86.500068,45.081288
3,1953-04-01,-24.631953,-93.802059,34.459801
4,1953-05-01,-3.780761,-62.245878,63.112391
...,...,...,...,...
844,2023-05-01,36.128572,-26.968662,96.385861
845,2023-06-01,26.645665,-36.686014,90.840766
846,2023-07-01,26.346136,-40.792180,81.448388
847,2023-08-01,21.536844,-42.221708,81.732442


In [31]:
# plot interactive chart
fig2 = plot_plotly(m2, forecast2)
fig2.write_html("temp.html")
fig2